****************************************************************************
* @file: DNN.py
* @project: ROBOFEI-HT - FEI 😛
* @author: Vinicius Nicassio Ferreira
* @version: V0.0.1
* @created: 23/10/2017
* @e-mail: vinicius.nicassio@gmail.com
* @brief: Class DNN
****************************************************************************

In [1]:
#ini-iPython

# Executando no diretório principal

In [2]:
import os
os.chdir('../../') #Executando na pasta Vision
os.system("cd Workbench/ && ./zerar.sh && cd ..")
os.system("cd Workbench/iPython/ && ./convertendo.sh && cd ../../")
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [3]:
# The standard libraries used in the vision system.
import tarfile # Used for manipulating tar files.
import pandas as pd # 

In [4]:
# The standard libraries used in the visual memory system.
import cv2 # OpenCV library used for image processing.
import numpy as np # Library used for operations with matrix and array.

# Libraries to supporting the network execution using Tensorflow.
#ini-iPython - Utilizado para o iPython reconhecer as classes
sys.path.append('/home/vinicius/models/research')
sys.path.append('/home/vinicius/models/research/object_detection')
#end-iPython
import tensorflow as tf
from utils import label_map_util
from utils import visualization_utils as vis_util

In [5]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe BasicThread como instanciação
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/BasicThread.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/BasicThread.py")
#end-iPython
from BasicThread import * # Base class with primary functions

# Class DNN
.
doxygen-iPython

In [6]:
#des-iPython class DNN(BasicThread):
#ini-tab -> Iniciando código classe

---- Variables ----

In [7]:
#ini-iPython

# ------- Lista de funções -------

#self-iPython __unzipNetwork
#self-iPython __instantiatingDNNVariables
#self-iPython __detectDNN
#self-iPython detect

#ini-iPython
class argumentos:
    camera = False
    dnn = True

a = argumentos()
#end-iPython

#end-iPython - Fim listas

# __EXTRACTION_DIRECTORY
.
doxygen-iPython

In [8]:
__EXTRACTION_DIRECTORY = "./Data/Rede" #self-iPython __EXTRACTION_DIRECTORY

# __PATH_TO_CKPT
Path to frozen detection graph. This is the actual model that is used for the object detection.
doxygen-iPython

In [9]:
__PATH_TO_CKPT = "frozen_inference_graph.pb" #self-iPython __PATH_TO_CKPT

# __PATH_TO_LABELS
List of the strings that is used to add correct label for each box.
doxygen-iPython

In [10]:
__PATH_TO_LABELS = "object-detection.pbtxt" #self-iPython __PATH_TO_LABELS

# __numclasses
The Number of classes that are detected by the network.
doxygen-iPython

In [11]:
__numclasses = None #self-iPython __numclasses

# __detection_graph
.
doxygen-iPython

In [12]:
__detection_graph = None #self-iPython __detection_graph

# __sess
.
doxygen-iPython

In [13]:
__sess = None #self-iPython __sess

# __label
.
doxygen-iPython

In [14]:
__label = None #self-iPython __label

# __parameters
.
doxygen-iPython

In [15]:
__parameters = None #self-iPython __parameters

In [16]:
imagetensor = detectionboxes = detectionscores = detectionclasses = numdetections = category_index = None
#ini-iPython
#self-iPython imagetensor
#self-iPython detectionboxes
#self-iPython detectionscores
#self-iPython detectionclasses
#self-iPython numdetections
#self-iPython category_index
#end-iPython

# __unzipNetwork
.
doxygen-iPython

In [17]:
#eself-iPython __unzipNetwork
#edes-iPython def __unzipNetwork(self, name):
def __unzipNetwork(name):
    tar = tarfile.open("./Data/"+name+".tar.gz")
    tar.extractall(path=__EXTRACTION_DIRECTORY)
    tar.close() #self-iPython __unzipNetwork

In [18]:
#ini-iPython

## Teste __unzipNetwork

In [19]:
__unzipNetwork("rede")
#end-iPython

IOError: [Errno 2] No such file or directory: './Data/rede.tar.gz'

# instantiatingDNNVariables
Used to create/instantiate the variables that will be used by DNN in detecting objects.
doxygen-iPython

In [ ]:
#eself-iPython __instantiatingDNNVariables
#edes-iPython def __instantiatingDNNVariables(self):
def __instantiatingDNNVariables( ):
    global __numclasses, __detection_graph, __sess, __label #debug-iPython
    global imagetensor, detectionboxes, detectionscores, detectionclasses, numdetections, category_index #debug-iPython
    # Reading network file.
    __detection_graph = tf.Graph()
    with __detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(__EXTRACTION_DIRECTORY+ "/" +__PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    label_map = label_map_util.load_labelmap(__EXTRACTION_DIRECTORY+ "/" +__PATH_TO_LABELS)
    
    filelabel = str(label_map).replace("\n", "").replace(" ", "").replace("{", "").replace("}", "")
    __label = {}
    for text in filelabel.split("item")[1:]:
        key = float(text[-1])
        name = text.split("\"")[1]
        __label[key] = name
    
    if self._args.dnn == True:
        __numclasses = str(label_map).count("id")
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=__numclasses, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)

    # Creating a section to run the detection.
    with __detection_graph.as_default():
        __sess = tf.Session(
            graph=__detection_graph,
            config=tf.ConfigProto(
                intra_op_parallelism_threads=1,
                inter_op_parallelism_threads=1
            )
        )
        
        imagetensor = __detection_graph.get_tensor_by_name('image_tensor:0')
        detectionboxes = __detection_graph.get_tensor_by_name('detection_boxes:0')
        detectionscores = __detection_graph.get_tensor_by_name('detection_scores:0')
        detectionclasses = __detection_graph.get_tensor_by_name('detection_classes:0')
        numdetections = __detection_graph.get_tensor_by_name('num_detections:0') #self-iPython __instantiatingDNNVariables

In [ ]:
#ini-iPython

## Teste __instantiatingDNNVariables

In [ ]:
self = BasicThread(a, "DNN", "Parameters")
__instantiatingDNNVariables()
__numclasses, __detection_graph, __sess

In [ ]:
__numclasses = __detection_graph = __sess = self = None
#end-iPython

# Constructor Class
doxygen-iPython

In [ ]:
#des-iPython def __init__(self, a):
#ini-tab -> Criando construtor
#edes-iPython super(DNN, self).__init__(a, "DNN", "Parameters")
self = BasicThread(a, "DNN", "Parameters")

# Creating and updating default parameter values.
__parameters = {
    "network_name": "rede",
    "threshold_to_train": 0.2,
}
__parameters = self._conf.readVariables(__parameters)

# Creating neural network.
__unzipNetwork(__parameters["network_name"])
__instantiatingDNNVariables()

In [ ]:
#end-tab -> Finalizando construtor

In [ ]:
#ini-iPython

## Teste construtor

In [ ]:
__numclasses, __detection_graph, __sess
#end-iPython

# detectDNN
.
doxygen-iPython

In [ ]:
#eself-iPython __detectDNN
#edes-iPython def __detectDNN(self, img):
def __detectDNN(img):
    global __sess #debug-iPython
    # Actual detection.
    image_np_expanded = np.expand_dims(img, axis=0)
    (boxes, scores, classes, num) = __sess.run(
        [detectionboxes, detectionscores, detectionclasses, numdetections],
        feed_dict={imagetensor: image_np_expanded}
    )
    
    if self._args.dnn == True:
        # Visualization of the results of a detection.
        img = cv2.resize(
            img,
            None,
            fx=380.0/img.shape[0],
            fy=380.0/img.shape[0]
        )
        vis_util.visualize_boxes_and_labels_on_image_array(
            img,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=2,
            min_score_thresh=__parameters["threshold_to_train"],
        )
    
    # Creating DataFrame detection.
    classes = classes.tolist()
    for i in xrange(len(classes[0])):
        classes[0][i] = __label[classes[0][i]]
    
    df = pd.DataFrame()
    df["classes"] = classes[0]
    df["scores"] = scores[0]
    df["boxes"] = boxes[0].tolist()
    
    return df, img #self-iPython __detectDNN

In [ ]:
#ini-iPython

## Teste detectDNN

In [ ]:
img = cv2.imread("/home/vinicius/objectDetect_clear/Train/imagenstreino/frames0_040.jpg")
df, img = __detectDNN(img)
cv2.imshow("Frame 2", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
df
#end-iPython

# detect
.
doxygen-iPython

In [ ]:
#eself-iPython detect
#edes-iPython def detect(self, observation):
def detect(observation):
    objects, observation['frame'] = __detectDNN(observation['frame'])


    






observation = {}

observation['frame'] = cv2.imread("/home/vinicius/objectDetect_clear/Train/imagenstreino/frames0_040.jpg")


detect(observation)








#self-iPython detect

In [ ]:
#ini-iPython

## Teste detect

In [ ]:

#end-iPython

In [ ]:
#end-tab -> Finalizando classe

In [ ]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [ ]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [ ]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [ ]:
#ini-iPython

## Teste nova_funcao

In [ ]:

#end-iPython

In [ ]:
#end-iPython